# IBM Cloud Pak for Data Data Virtualization Lab Setup

### Where to find this notebook online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/CPDDVLAB.

### What is notebook does
This notebook is a operational guide to prepare for a digital bootcamp or hands on lab for a team of people. It uses an existing IBM Cloud Pak for Data Cluster that has Data Connections as we as virtual tables and views created for the admin user. 

All the code in this notebook runs using the main **admin** userid.

The notebook includes instructions on how to complete some steps directly in the IBM Cloud Pak for Data console. It also includes Python code that creates users, manages privledges, and tests that all the required tables and views work.

The notebooks includes three sections:
1. Utility python routines that can be used to check the status of the system
2. Setup of the main project, creation of users, granting privledges, and testing tables and views
3. Teardown, including deleting any user created objects, revoking privledges, deleting users and the main project

#### RESTful Services
IBM Cloud Pak for Data is built on a set of microservices that communicate with each other and the Console user interface using RESTful APIs. You can use these services to automate anything you can do throught the user interface.

This Jupyter Notebook contains examples of how to use the Open APIs to retrieve information from the virtualization service, how to run SQL statements directly against the service through REST and how to provide authoritization to objects. This provides a way write your own script to automate the setup and configuration of the virtualization service.

The next part of the lab relies on a set of base classes to help you interact with the RESTful Services API for IBM Cloud Pak for Data Virtualization. You can access this library on GITHUB. The commands below download the library and run them as part of this notebook.
<pre>
&#37;run CPDDVRestClass.ipynb
</pre>
The cell below loads the RESTful Service Classes and methods directly from GITHUB. Note that it will take a few seconds for the extension to load, so you should generally wait until the "Db2 Extensions Loaded" message is displayed in your notebook. 
1. Click the cell below
2. Click **Run**

In [1]:
!wget -O CPDDVRestClass.ipynb https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/CPDDVRestClass.ipynb
%run CPDDVRestClass.ipynb

--2020-04-15 14:39:31--  https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/CPDDVRestClass.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15880 (16K) [text/plain]
Saving to: ‘CPDDVRestClass.ipynb’

CPDDVRestClass.ipyn 100%[===================>]  15.51K  --.-KB/s    in 0.1s    

2020-04-15 14:39:32 (108 KB/s) - ‘CPDDVRestClass.ipynb’ saved [15880/15880]



## Establishing a Connection to the Console

### Connections
To connect to the Data Virtualization service you need to provide the URL, the service name (v1) and profile the console user name and password. The next cell connects to the console from inside the IBM CPD Cluster.

In [2]:
# Connect to the Db2 Data Management Console service

# From Outside the Cluster
Console  = 'https://services-uscentral.skytap.com:9152'
# From Inside the Cluster
# Console  = 'https://openshift-skytap-nfs-lb.ibm.com'
user     = 'admin'
password = 'DTE.cp4data!'

# Set up the required connection
CPDAPI = Db2(Console)
api = '/v1'
CPDAPI.authenticate(api, user, password)
database = Console

Token Retrieved


## Utility Routines

### Data Sources
The next cell calls a restful service that displays all the currently configured data connections for data virtualization.

The following Python function (getDataSources) runs SQL against the **QPLEXSYS.LISTRDB** catalog table and combines it with a stored procedure call **QPLEXSYS.LISTRDBCDETAILS()** to add the **AVAILABLE** column to the results. The IBM Cloud Pak for Data Virtualization Service checks each data sources every 5 to 10 seconds to ensure that it is still up and available. In the table (DataFrame) in the next cell a **1** in the **AVAILABLE** column indicates that the data source is responding. A **0** indicdates that it is not longer responding. 

In [3]:
# Display the Available Data Sources already configured
dataSources = CPDAPI.getDataSources()
display(dataSources)

,CID,USR,SRCTYPE,SRCHOSTNAME,SRCPORT,DBNAME,AVAILABLE
0,MONGO10213,admin,MongoDB,10.1.1.12,27017,mongo_onprem_stocks,1
1,DB210220,dvdemo,DB2,db2w-wvigssp.us-east.db2w.cloud.ibm.com,50001,bludb,1
2,INFOR10146,informix,Informix,10.1.1.12,26111,STOCKS,1
3,DB210263,db2inst1,DB2,52.70.186.131,31593,bludb,1
4,DVM10060,ssibm59,DVM,rs01.rocketsoftware.com,15082,SQL92,1
5,POSTG10262,psdte,PostgreSQL,23.100.82.91,5432,psazdb,1
6,DB210121,user999,DB2,openshift-skytap-nfs-woker-5.ibm.com,31928,BLUDB,1
7,DB210261,db2inst1,DB2,23.100.82.91,31574,azdb,1


#### Run SQL through the SQL Editor Service
You can also use the SQL Editor service to run your own SQL. Statements are submitted to the editor. Your code then needs to poll the editor service until the script is complete. Fortunately you can use the DB2 class included in this lab so that it becomes a very simple Python call. The **runScript** routine runs the SQL and the **displayResults** routine formats the returned JSON. 

In [4]:
CPDAPI.displayResults(CPDAPI.runScript('SELECT * FROM TRADING.MOVING_AVERAGE'))

Statement: 0: SELECT * FROM TRADING.MOVING_AVERAGE
Runtime ms: 273.99998903274536


,WEEK,OPEN,MOVING_AVG
0,51,174.87,172.977500000000000000000000
1,51,174.17,173.267058823529411764705882
2,51,174.68,173.427222222222222222222222
3,52,170.80,173.740000000000000000000000
4,52,170.10,173.983500000000000000000000
5,52,171.00,174.141428571428571428571428
6,52,170.52,174.285000000000000000000000
7,1,170.16,174.413913043478260869565217
8,1,172.53,174.417916666666666666666666
9,1,172.54,174.321200000000000000000000


### Virtualized Tables and Views
The next two cells are useful to determine all the virtualized data availble to the admin user and the objects available by role.

In [ ]:
# Display the Virtualized Assets Avalable to Engineers and Users
roles = ['DV_ENGINEER','DV_USER']
for role in roles:
    r = CPDAPI.getRole(role)
    if (CPDAPI.getStatusCode(r)==200):
        json = CPDAPI.getJSON(r)
        df = pd.DataFrame(json_normalize(json['objects']))
        print(', '.join(list(df)))
        display(df)
    else:
        print(CPDAPI.getStatusCode(r))  

In [ ]:
### Display All Virtualized Tables and Views
display(CPDAPI.getVirtualizedTablesDF())
display(CPDAPI.getVirtualizedViewsDF())

### Cloud Pak for Data User Management
The next two cells can be used to list existing CPD users and add a new user to the system.

In [ ]:
# Get the list of CPD Users
r = CPDAPI.getUsers()
if (CPDAPI.getStatusCode(r)==200):
    json = CPDAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print(', '.join(list(df)))
    display(df[['uid','username','displayName']])
else:
    print(CPDAPI.getStatusCode(r))

In [ ]:
# Add a Single user to CPD
username = "LABUSER1"
displayName = "LABUSER1"
email = "kohlmann@ca.ibm.com"
user_roles = ["Data Scientist"]
password = 'password'
r = CPDAPI.addUser(username, displayName, email, user_roles, password)
if (CPDAPI.getStatusCode(r)==201):
    print('User Added')
else:
    print(CPDAPI.getStatusCode(r))

## Lab Setup

### Creating the Data Virtualization Project
The first step in setting up the lab is to create a project that all the lab users can share. You will then add a single hands on lab notebook to the project and finally make a copy for each participant.

###  Create the Data Virtualization Project
1. Right-click the following link and select **open link in new window** to open the IBM Cloud Pak for Data Console: https://services-uscentral.skytap.com:9152/
2. Organize your screen so that you can see both this notebook as well as the IBM Cloud Pak for Data Console at the same time. This will make it much easier for you to complete the lab without switch back and forth between screens.
3. Sign in using the **admin** userid and password
4. Click the three bar main navigation menu

    <img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/2.42.03 Three Bar.png">
    
5. Select **Projects**    
    
    <img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/11.17.07 Projects.png">
6. Select **+ New project**
7. Select **Analytics project**
8. Click **OK**
9. Click **Create an empty project** (you may have to click twice)
10. Enter **Data Virtualization Hands on Lab** as the Project name
11. Click **Create**

 
###  Add the template notebook to the Project 
 
1. From the Projects list, click **Data Virtualization Hands on Lab**

3. From the My Projects screen click **Add to project** at the top right of the screen

    <img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/11.17.48 Add to project.png">
    
4. Click **Notebook**

    <img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/11.17.59 Notebook.png">

5. Click **From URL**
6. Enter **DV Lab** in the Notebook **Name** field
7. Copy and paste the following link into the **Notebook URL** field:
    https://github.com/Db2-DTE-POC/CPDDVLAB/blob/master/CPD-DV%20Hands%20on%20Lab%20Preloaded.ipynb
8. Add an optional description

    <img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/11.25.23 New notebook.png">

9. Click **Create Notebook**

### Duplicate the template notebook
1. Click **Data Virtualization Hands on Lab** to navigate back to the list of assets
2. Scroll down until you see the new notebook **DB Lab** listed
3. Click the **elipsis icon** at the far right of the DV Lab
4. Click **duplicate**
5. **Repeat steps 3 and 4** nine more times. **Make sure to always select the original template notebook**

### Add Users to CPD and Data Virtualization

In [ ]:
# Add x Data Scientists, LABUSERS to CPD

ids = 10
userList = {'UserRoot':['LABUSER','LABDATAENGINEER'],'Role':[['Data Scientist','Developer'],['Data Engineer']]}
userListDF = pd.DataFrame(userList) 
email = 'kohlmann@ca.ibm.com'
password = 'dtedvlab'

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        username = userListDF['UserRoot'].iloc[row]+str(x)
        user_role = userListDF['Role'].iloc[row]
        displayName = username
        r = CPDAPI.addUser(username, displayName, email, user_role, password)
        if (CPDAPI.getStatusCode(r)==201):
            print('User: '+username+' Added as a '+str(user_role))
        else:
            print(CPDAPI.getStatusCode(r))

In [ ]:
# Add x Users and Engineers to the DV Service
ids = 10
userList = {'UserRoot':['LABUSER','LABDATAENGINEER'],'Role':['User','Engineer']}
userListDF = pd.DataFrame(userList) 

df = CPDAPI.getUsersDF() # Get existing list of users to get the uid

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        display_name = userListDF['UserRoot'].iloc[row]+str(x)
        role = userListDF['Role'].iloc[row]
        
        r = CPDAPI.addUserToDV(display_name, role, df)
        if (CPDAPI.getStatusCode(r)==200):
            print('User: '+display_name+' added to Data Virtualization Service')
        else:
            print(CPDAPI.getStatusCode(r))

### Grant Access to Data Engineers to Existing Views and Tables

In [ ]:
# Grant Access to Data Engineers to all the Views owned by the logged in user
ViewsDF = CPDAPI.getVirtualizedViewsDF()
roleToGrant = 'DV_ENGINEER'
for index, row in ViewsDF.iterrows():
    name = row['viewname']
    schema = row['viewschema']

    r = CPDAPI.grantPrivledgeToRole(name, schema, roleToGrant)
    if (CPDAPI.getStatusCode(r)==200):
        print('Access granted')
    else:
        print(CPDAPI.getStatusCode(r))

In [ ]:
# Grant Access to Data Engineers to all the Virtualizated Tables owned by the logged in user
TablesDF = CPDAPI.getVirtualizedTablesDF()
roleToGrant = 'DV_ENGINEER'
for index, row in TablesDF.iterrows():
    name = row['table_name']
    schema = row['table_schema']

    r = CPDAPI.grantPrivledgeToRole(name, schema, roleToGrant)
    if (CPDAPI.getStatusCode(r)==200):
        print('Access granted')
    else:
        print(CPDAPI.getStatusCode(r))

### Test Existing Data Sources

In [ ]:
# Display the Available Data Sources already configured
dataSources = CPDAPI.getDataSources()
display(dataSources)

### Test Existing Virtualized Tables, Folded Tables and Views

In [5]:
# Test that the existing views all work
sqlText = \
'''
-- zOS VSAM
SELECT COUNT(*) FROM DVDEMO.STOCK_SYMBOLS;
SELECT * FROM DVDEMO.STOCK_SYMBOLS FETCH FIRST 2 ROWS ONLY;

-- Folded Virtual Tables
SELECT COUNT(*) FROM FOLDING.STOCK_HISTORY;
SELECT * FROM FOLDING.STOCK_HISTORY FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM FOLDING.ACCOUNTS_DV;
SELECT * FROM FOLDING.ACCOUNTS_DV FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM FOLDING.STOCK_TRANSACTIONS_DV;
SELECT * FROM FOLDING.STOCK_TRANSACTIONS_DV FETCH FIRST 2 ROWS ONLY;

-- Mongo DB
SELECT COUNT(CUSTOMER_ID) FROM MONGO_ONPREM.CUSTOMER_CONTACT;
SELECT * FROM MONGO_ONPREM.CUSTOMER_CONTACT FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(CUSTOMER_ID) FROM MONGO_ONPREM.CUSTOMER_IDENTITY;
SELECT * FROM MONGO_ONPREM.CUSTOMER_IDENTITY FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(CUSTOMER_ID) FROM MONGO_ONPREM.CUSTOMER_PAYMENT;
SELECT * FROM MONGO_ONPREM.CUSTOMER_PAYMENT FETCH FIRST 2 ROWS ONLY;

-- Views
SELECT COUNT(*) FROM TRADING.MOVING_AVERAGE;
SELECT * FROM TRADING.MOVING_AVERAGE FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM TRADING.VOLUME;
SELECT * FROM TRADING.VOLUME FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM TRADING.THREEPERCENT;
SELECT * FROM TRADING.THREEPERCENT FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM TRADING.TRANSBYCUSTOMER;
SELECT * FROM TRADING.TRANSBYCUSTOMER FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM TRADING.TOPBOUGHTSOLD;
SELECT * FROM TRADING.TOPBOUGHTSOLD FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM TRADING.TOPFIVE;
SELECT * FROM TRADING.TOPFIVE FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM TRADING.BOTTOMFIVE;
SELECT * FROM TRADING.BOTTOMFIVE FETCH FIRST 2 ROWS ONLY;
SELECT COUNT(*) FROM TRADING.OHIO;
SELECT * FROM TRADING.OHIO FETCH FIRST 2 ROWS ONLY;
'''

CPDAPI.displayResults(CPDAPI.runScript(sqlText))

Statement: 0: -- zOS VSAM
SELECT COUNT(*) FROM DVDEMO.STOCK_SYMBOLS
Runtime ms: 559.0000152587891


,1
0,31



Statement: 1: SELECT * FROM DVDEMO.STOCK_SYMBOLS FETCH FIRST 2 ROWS ONLY
Runtime ms: 651.0000228881836


,SYMBOL,COMPANY
0,AAPL,Apple Inc.
1,AXP,American Express Company



Statement: 1: SELECT * FROM DVDEMO.STOCK_SYMBOLS FETCH FIRST 2 ROWS ONLY
Runtime ms: 651.0000228881836


...

,CUSTID,BALANCE
0,103333,39871.79
1,103334,-3966.53



Statement: 28: SELECT COUNT(*) FROM TRADING.OHIO
Runtime ms: 11916.999816894531


,1
0,49



Statement: 29: SELECT * FROM TRADING.OHIO FETCH FIRST 2 ROWS ONLY
Runtime ms: 1409.999966621399


,LASTNAME,TOTAL
0,Boone,2098.25
1,Burgess,1565.16


## Lab Teardown

### Drop the Existing Project

### Remove Tables and Views Created by Lab Participants

In [ ]:
# Delete Virtualized Tables Created by Lab Participants

virtualTables = CPDAPI.getVirtualizedTablesDF()
virtualUserTables = virtualTables.loc[virtualTables['owner'] != 'USER999']
display(virtualUserTables)
for index, row in virtualUserTables.iterrows():
    schema = row['table_schema']
    table = row['table_name']
    source = row['data_source_table_name']
    CPDAPI.deleteVirtualizedTable(schema, table, source)
    if (CPDAPI.getStatusCode(r)==200):
        print('Virtualized Table deleted')
    else:
        print(CPDAPI.getStatusCode(r))
display(CPDAPI.getVirtualizedTablesDF())

In [ ]:
# Delete Virtualized Views Created by Lab Participants

views = CPDAPI.getVirtualizedViewsDF()
userViews = views.loc[views['owner'] != 'USER999']
display(userViews)
for index, row in userViews.iterrows():
    schema = row['viewschema']
    view = row['viewname']
    CPDAPI.deleteView(schema, view)
    if (CPDAPI.getStatusCode(r)==200):
        print('View deleted')
    else:
        print(CPDAPI.getStatusCode(r))
display(CPDAPI.getVirtualizedViewsDF())

### Remove Users from Data Virtualization Server and CPD

In [ ]:
# Drop x users and engineers from the DV Service
ids = 10
userList = {'UserRoot':['LABUSER','LABDATAENGINEER']}
userListDF = pd.DataFrame(userList) 

df = CPDAPI.getUsersDF() # Get existing list of users to get the uid

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        display_name = userListDF['UserRoot'].iloc[row]+str(x)
        
        r = CPDAPI.dropUserFromDV(display_name, df)
        if (CPDAPI.getStatusCode(r)==200):
            print('User: '+display_name+' dropped from Data Virtualization Service')
        else:
            print(CPDAPI.getStatusCode(r))

In [ ]:
# Drop x users and engineers from CPD
ids = 10
userList = {'UserRoot':['labuser','labdataengineer']}
userListDF = pd.DataFrame(userList) 

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        username = userListDF['UserRoot'].iloc[row]+str(x)

        r = CPDAPI.dropUser(username)
        if (CPDAPI.getStatusCode(r)==200):
            print('User: '+username+' Dropped')
        else:
            print(CPDAPI.getStatusCode(r))

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]